In [0]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
csvfile = 'drive/My Drive/Colab Notebooks/Travel Pony Facebook.csv'
df=pd.read_csv(csvfile)

In [4]:
df.head(3)

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.0,3,246,243,0,0,0,0,243,100.0,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.0,6,215,209,0,0,0,0,209,50.0,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.0,2,252,250,0,0,0,0,250,150.0,1.190476


In [0]:
df['cost per impression']=df['Amount Spent (USD)']/df['Impressions']

In [6]:
!pip install arrow

     |████████████████████████████████| 51kB 1.5MB/s 


In [0]:
import arrow as arrow

In [17]:
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['Day of Week'] = df['Start Date'].dt.day_name()
df.columns
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),cost per impression,Day of Week
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0.000466,Monday
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,0.000652,Sunday
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,0.000448,Saturday
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,0.000452,Friday
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,0.002464,Friday


In [22]:
df.groupby('Day of Week', as_index=False).agg({'cost per impression': 'mean'})

,Day of Week,cost per impression
0,Friday,0.004097
1,Monday,0.002910
2,Saturday,0.002629
3,Sunday,0.003688
4,Thursday,0.003398
5,Tuesday,0.003040
6,Wednesday,0.002981


Saturday has the lowest cost per impression and is therefore the day of the week that works best. Friday has the highest cost per impression and therefore is the worst day.


In [23]:
df.corr()

,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),cost per impression
Reach,1.000000,0.334101,0.966851,0.709466,0.722249,0.703124,0.304388,0.597648,0.605244,0.151727,0.100254,0.153288,0.205678,0.858860,0.646366,0.168697,0.028005
Frequency,0.334101,1.000000,0.386949,0.128473,0.135103,0.130201,0.000182,0.111645,0.119780,-0.015511,-0.008028,-0.012113,-0.009054,0.240826,0.154855,0.043232,-0.018217
Impressions,0.966851,0.386949,1.000000,0.538032,0.553394,0.532913,0.158246,0.459510,0.474330,0.065275,0.043358,0.067018,0.090544,0.769145,0.565503,0.091639,0.006973
Clicks,0.709466,0.128473,0.538032,1.000000,0.998287,0.883411,0.591499,0.797945,0.786167,0.465285,0.242256,0.407737,0.448348,0.817139,0.594057,0.209975,0.088315
Unique Clicks,0.722249,0.135103,0.553394,0.998287,1.000000,0.882993,0.584614,0.787337,0.775598,0.445915,0.224601,0.379449,0.432454,0.822962,0.597671,0.213844,0.086798
Amount Spent (USD),0.703124,0.130201,0.532913,0.883411,0.882993,1.000000,0.757612,0.819253,0.789348,0.386383,0.259781,0.420018,0.479713,0.817490,0.637906,0.323046,0.119201
Page Likes,0.304388,0.000182,0.158246,0.591499,0.584614,0.757612,1.000000,0.650871,0.579933,0.454491,0.290983,0.432091,0.402059,0.477790,0.304196,0.271398,0.143931
Page Engagement,0.597648,0.111645,0.459510,0.797945,0.787337,0.819253,0.650871,1.000000,0.995945,0.700280,0.387753,0.648593,0.814885,0.731632,0.617181,0.147983,0.117141
Post Engagement,0.605244,0.119780,0.474330,0.786167,0.775598,0.789348,0.579933,0.995945,1.000000,0.697600,0.381613,0.644780,0.826786,0.728476,0.626231,0.126639,0.108649
Post Likes,0.151727,-0.015511,0.065275,0.465285,0.445915,0.386383,0.454491,0.700280,0.697600,1.000000,0.424246,0.689834,0.639658,0.240632,0.267482,0.058336,0.106425


In [25]:
print(df[['Amount Spent (USD)','Reach','Frequency','Unique Clicks','Page Likes']].corr())

                    Amount Spent (USD)     Reach  ...  Unique Clicks  Page Likes
Amount Spent (USD)            1.000000  0.703124  ...       0.882993    0.757612
Reach                         0.703124  1.000000  ...       0.722249    0.304388
Frequency                     0.130201  0.334101  ...       0.135103    0.000182
Unique Clicks                 0.882993  0.722249  ...       1.000000    0.584614
Page Likes                    0.757612  0.304388  ...       0.584614    1.000000

[5 rows x 5 columns]


The correlation that is the strongest with Amount Spent (USD) is Unique Clicks at .883. This means that these two variables have a strong relationship with each other. When Amount Spent changes (increases or decreases), Unique Clicks is likely to follow the same pattern because they are highly correlated.

In [0]:
import statsmodels.api as sm

In [15]:
target=df['Unique Clicks']
indep= df[['Reach','Frequency']]
model= sm.OLS(target, indep).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Thu, 14 Nov 2019   Prob (F-statistic):                        0.00
Time:                        00:54:35   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The variable that most strongly predicts unique clicks is Reach because of its low p-value and extremely high t value compared to the t value of Frequency. Although, Frequency is still a significant predictor of unique clicks.
